In [1]:
# Change the CWD to be the root so that the giles_conf.json file is found in the correct location
import os
os.chdir("../../../")

In [2]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

In [3]:
import pandas as pd
import emission.core.get_database as edb
import bson.objectid as boi
import numpy as np
import scipy.interpolate as spi
import arrow

In [4]:
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.builtin_timeseries as estb
import emission.core.wrapper.entry as ecwe
import emission.core.wrapper.location as ecwl
import emission.analysis.intake.cleaning.location_smoothing as eaicl

In [5]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

In [6]:
edb.get_timeseries_db().distinct('user_id')

[UUID('079e0f1a-c440-3d7c-b0e7-de160f748e35'),
 UUID('d1ebc47a-4180-4464-9f83-dabc36c37e39'),
 UUID('0763de67-f61e-3f5d-90e7-518e69793954')]

In [7]:
curr_id = edb.get_timeseries_db().distinct('user_id')[0]
trips_df = esda.get_data_df(esda.RAW_TRIP_KEY, curr_id, time_query=None)

In [8]:
trips_df[["_id", "start_fmt_time", "start_ts", "end_fmt_time", "end_ts", "start_place"]]

,_id,start_fmt_time,start_ts,end_fmt_time,end_ts,start_place
0,57bfc294f6858f9f05b2b2eb,2016-02-22T13:06:06.142961-08:00,1456175166.142961,2016-02-22T13:13:19.001847-08:00,1456175599.001847,57bfc294f6858f9f05b2b2ea
1,57bfc294f6858f9f05b2b2ed,2016-02-22T15:33:18.550963-08:00,1456183998.550963,2016-02-22T15:37:12.000136-08:00,1456184232.000136,57bfc294f6858f9f05b2b2ec
2,57bfc294f6858f9f05b2b2ef,2016-02-22T15:49:43.999972-08:00,1456184983.999972,2016-02-22T16:04:00.997765-08:00,1456185840.997765,57bfc294f6858f9f05b2b2ee
3,57bfc294f6858f9f05b2b2f1,2016-02-22T16:49:30.999990-08:00,1456188570.999990,2016-02-22T16:56:58.000047-08:00,1456189018.000047,57bfc294f6858f9f05b2b2f0
4,57bfc294f6858f9f05b2b2f3,2016-02-22T19:41:42.085297-08:00,1456198902.085297,2016-02-22T19:48:35.000033-08:00,1456199315.000033,57bfc294f6858f9f05b2b2f2
5,57bfc294f6858f9f05b2b2f5,2016-02-22T19:58:50.000111-08:00,1456199930.000111,2016-02-22T20:12:15.000095-08:00,1456200735.000095,57bfc294f6858f9f05b2b2f4


In [9]:
import emission.storage.decorations.trip_queries as esdt

We are going to focus on the trips highlighted in the bug https://github.com/e-mission/e-mission-server/issues/288

## Short trips highlighted in the bug

### Trip from Kaiser to Cognition (\_id = 57bb7d6af6858f2284bb4682)

In [10]:
kctrip = trips_df.iloc[1] 

In [11]:
kcid = kctrip._id

In [12]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, kcid)[0]

In [13]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.0848234194634, 37.40312794587273], "type": "Point"},
 u'2016-02-22T15:33:18.550963-08:00')

In [14]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [15]:
# loc_entry_df.append(loc_entry_df).reset_index()[["fmt_time", "ts"]]

In [16]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
[(e.data.ts, e.data.fmt_time, e.data.loc.coordinates) for e in loc_entry_list]

[(1456183998.550963,
  u'2016-02-22T15:33:18.550963-08:00',
  [-122.0848234194634, 37.40312794587273]),
 (1456184136.000126,
  u'2016-02-22T15:35:36.000126-08:00',
  [-122.0830013976638, 37.3962290492257]),
 (1456184145.000135,
  u'2016-02-22T15:35:45.000135-08:00',
  [-122.0832050779109, 37.39579197488482]),
 (1456184185.000135,
  u'2016-02-22T15:36:25.000135-08:00',
  [-122.0836216584985, 37.39545192107315]),
 (1456184191.000135,
  u'2016-02-22T15:36:31.000135-08:00',
  [-122.0838668291663, 37.39493111151959]),
 (1456184196.000135,
  u'2016-02-22T15:36:36.000135-08:00',
  [-122.0840954036658, 37.39451838660742]),
 (1456184201.000136,
  u'2016-02-22T15:36:41.000136-08:00',
  [-122.0843451005612, 37.39408709577973]),
 (1456184205.000135,
  u'2016-02-22T15:36:45.000135-08:00',
  [-122.0845944621806, 37.39364503420646]),
 (1456184209.000136,
  u'2016-02-22T15:36:49.000136-08:00',
  [-122.0848776028697, 37.39320812750365]),
 (1456184213.000134,
  u'2016-02-22T15:36:53.000134-08:00',
  [-1

#### 2-D interpolation

In [17]:
if len(loc_entry_list) < 4:
    loc_entry_list.extend(loc_entry_list)

In [18]:
len(loc_entry_list)

14

In [19]:
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)

In [20]:
time_fn = spi.interp2d(np.array(loc_entry_df.latitude), np.array(loc_entry_df.longitude), np.array(loc_entry_df.ts))

/Users/shankari/OSS/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:975: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=1,1 nx,ny=6,6 m=14 fp=3.023549 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [21]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, trips_df.iloc[1].start_place)

In [22]:
start_place

Entry({u'data': {u'enter_fmt_time': u'2016-02-22T13:13:19.001847-08:00', u'exit_local_dt': {u'hour': 15, u'month': 2, u'second': 18, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 33}, u'exit_fmt_time': u'2016-02-22T15:33:18.550963-08:00', u'enter_local_dt': {u'hour': 13, u'month': 2, u'second': 19, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 13}, u'ending_trip': ObjectId('57bfc294f6858f9f05b2b2eb'), u'starting_trip': ObjectId('57bfc294f6858f9f05b2b2ed'), u'source': u'DwellSegmentationDistFilter', u'location': {u'type': u'Point', u'coordinates': [-122.0841170289759, 37.40358810875323]}, u'enter_ts': 1456175599.001847, u'duration': 8399.549115896225, u'exit_ts': 1456183998.550963}, u'_id': ObjectId('57bfc294f6858f9f05b2b2ec'), u'user_id': UUID('079e0f1a-c440-3d7c-b0e7-de160f748e35'), u'metadata': {u'write_fmt_time': u'2016-08-25T21:16:20.780295-07:00', u'write_ts': 1472184980.780295, u'time_zone': u

In [23]:
arrow.get(time_fn(start_place.data.location.coordinates[1], start_place.data.location.coordinates[0])).to("America/Los_Angeles")

<Arrow [2016-02-22T15:36:46.667928-08:00]>

In [24]:
arrow.get(time_fn(start_place.data.location.coordinates[0], start_place.data.location.coordinates[1])).to("America/Los_Angeles")

<Arrow [1969-12-31T16:00:00-08:00]>

#### Doesn't work!!

#### Use average time and distance to start to estimate travel time

In [25]:
import emission.analysis.intake.cleaning.clean_and_resample as eaicc
import emission.analysis.point_features as pf
import emission.core.common as ecc

In [26]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [27]:
with_speeds_df_nona = with_speeds_df.dropna()

In [28]:
with_speeds_df.speed.median()

10.431133232715087

In [29]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, kctrip.start_place)

In [30]:
[loc_entry_df.iloc[0].longitude, loc_entry_df.iloc[0].latitude], start_place.data.location.coordinates

([-122.0848234194634, 37.403127945872733],
 [-122.0841170289759, 37.40358810875323])

In [31]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

80.6933529753


In [32]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

7.735818455684122

In [33]:
(40 + add_time) / 60

0.7955969742614021

In [34]:
75 + add_dist

155.69335297533735

### Close enough to the real values (418m/3 mins)

### Trip from home to the bookstore (\_id = 57bb7d67f6858f2284bb465d)

In [35]:
hbtrip = trips_df.iloc[-2]

In [36]:
hbid = hbtrip._id

In [37]:
len(esdt.get_raw_sections_for_trip(curr_id, hbid))

1

In [38]:
for s in esdt.get_raw_sections_for_trip(curr_id, hbid):
    print s.data.start_fmt_time, s.data.end_fmt_time, s.data.sensed_mode, s.data.start_stop

2016-02-22T19:41:42.085297-08:00 2016-02-22T19:48:35.000033-08:00 MotionTypes.WALKING None


In [39]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, hbid)[0]

In [40]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.0826288376279, 37.39067786774222], "type": "Point"},
 u'2016-02-22T19:41:42.085297-08:00')

In [41]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [42]:
# loc_entry_df.append(loc_entry_df).reset_index()[["fmt_time", "ts"]]

In [43]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
[(e.data.ts, e.data.fmt_time, e.data.loc.coordinates) for e in loc_entry_list]

[(1456198902.085297,
  u'2016-02-22T19:41:42.085297-08:00',
  [-122.0826288376279, 37.39067786774222]),
 (1456198902.085297,
  u'2016-02-22T19:41:42.085297-08:00',
  [-122.0826288376279, 37.39067786774222]),
 (1456198905.00003,
  u'2016-02-22T19:41:45.000030-08:00',
  [-122.0828158223276, 37.39063345830645]),
 (1456198905.00003,
  u'2016-02-22T19:41:45.000030-08:00',
  [-122.0828158223276, 37.39063345830645]),
 (1456198969.000034,
  u'2016-02-22T19:42:49.000034-08:00',
  [-122.0823213738595, 37.39040337506439]),
 (1456198969.000034,
  u'2016-02-22T19:42:49.000034-08:00',
  [-122.0823213738595, 37.39040337506439]),
 (1456199001.000033,
  u'2016-02-22T19:43:21.000033-08:00',
  [-122.0821464435403, 37.39083759955819]),
 (1456199001.000033,
  u'2016-02-22T19:43:21.000033-08:00',
  [-122.0821464435403, 37.39083759955819]),
 (1456199034.000033,
  u'2016-02-22T19:43:54.000033-08:00',
  [-122.0819117502515, 37.39125740717854]),
 (1456199034.000033,
  u'2016-02-22T19:43:54.000033-08:00',
  [-12

#### 2-D interpolation

In [44]:
if len(loc_entry_list) < 4:
    loc_entry_list.extend(loc_entry_list)

In [45]:
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)

In [46]:
np.array(loc_entry_df.latitude), np.array(loc_entry_df.longitude), np.array(loc_entry_df.ts)

(array([ 37.39067787,  37.39063346,  37.39040338,  37.3908376 ,
         37.39125741,  37.39159641,  37.39153439,  37.39163539,
         37.39206379,  37.39235883]),
 array([-122.08262884, -122.08281582, -122.08232137, -122.08214644,
        -122.08191175, -122.08153373, -122.08095873, -122.08040267,
        -122.08021685, -122.07978786]),
 array([  1.45619890e+09,   1.45619891e+09,   1.45619897e+09,
          1.45619900e+09,   1.45619903e+09,   1.45619908e+09,
          1.45619916e+09,   1.45619920e+09,   1.45619923e+09,
          1.45619932e+09]))

In [47]:
len(np.array(loc_entry_df.latitude)), len(np.array(loc_entry_df.longitude)), len(np.array(loc_entry_df.ts))

(10, 10, 10)

In [48]:
time_fn = spi.interp2d(np.array(loc_entry_df.latitude), np.array(loc_entry_df.longitude), np.array(loc_entry_df.ts))

/Users/shankari/OSS/anaconda/lib/python2.7/site-packages/scipy/interpolate/fitpack.py:975: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=1,1 nx,ny=6,5 m=10 fp=75.833994 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [49]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, hbtrip.start_place)

In [50]:
start_place

Entry({u'data': {u'enter_fmt_time': u'2016-02-22T16:56:58.000047-08:00', u'exit_local_dt': {u'hour': 19, u'month': 2, u'second': 42, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 41}, u'exit_fmt_time': u'2016-02-22T19:41:42.085297-08:00', u'enter_local_dt': {u'hour': 16, u'month': 2, u'second': 58, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 56}, u'ending_trip': ObjectId('57bfc294f6858f9f05b2b2f1'), u'starting_trip': ObjectId('57bfc294f6858f9f05b2b2f3'), u'source': u'DwellSegmentationDistFilter', u'location': {u'type': u'Point', u'coordinates': [-122.0861964953338, 37.39118427507336]}, u'enter_ts': 1456189018.000047, u'duration': 9884.085250139236, u'exit_ts': 1456198902.085297}, u'_id': ObjectId('57bfc294f6858f9f05b2b2f2'), u'user_id': UUID('079e0f1a-c440-3d7c-b0e7-de160f748e35'), u'metadata': {u'write_fmt_time': u'2016-08-25T21:16:20.861673-07:00', u'write_ts': 1472184980.861673, u'time_zone': u

In [51]:
start_place.data.location

{"coordinates": [-122.0861964953338, 37.39118427507336], "type": "Point"}

In [52]:
arrow.get(time_fn(start_place.data.location.coordinates[1], start_place.data.location.coordinates[0])).to("America/Los_Angeles")

<Arrow [2016-08-08T10:59:21.083407-07:00]>

In [53]:
arrow.get(time_fn(start_place.data.location.coordinates[0], start_place.data.location.coordinates[1])).to("America/Los_Angeles")

<Arrow [1969-12-31T16:00:00-08:00]>

#### Doesn't work!! (19:41:02) is after the current trip start

#### Use average time and distance to start to estimate travel time

In [54]:
import emission.analysis.intake.cleaning.clean_and_resample as eaicc
import emission.analysis.point_features as pf
import emission.core.common as ecc

In [55]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456198902.085297,2016-02-22T19:41:42.085297-08:00,37.390678,-122.082629
1,1456198905.000030,2016-02-22T19:41:45.000030-08:00,37.390633,-122.082816
2,1456198969.000034,2016-02-22T19:42:49.000034-08:00,37.390403,-122.082321
3,1456199001.000033,2016-02-22T19:43:21.000033-08:00,37.390838,-122.082146
4,1456199034.000033,2016-02-22T19:43:54.000033-08:00,37.391257,-122.081912
5,1456199080.000033,2016-02-22T19:44:40.000033-08:00,37.391596,-122.081534
6,1456199158.000033,2016-02-22T19:45:58.000033-08:00,37.391534,-122.080959
7,1456199202.000034,2016-02-22T19:46:42.000034-08:00,37.391635,-122.080403
8,1456199231.000033,2016-02-22T19:47:11.000033-08:00,37.392064,-122.080217
9,1456199315.000033,2016-02-22T19:48:35.000033-08:00,37.392359,-122.079788


In [56]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [57]:
with_speeds_df.speed, with_speeds_df.sensed_speed

(0   0.000000
 1   5.915331
 2   0.790988
 3   1.584267
 4   1.547820
 5   1.094819
 6   0.657234
 7   1.145274
 8   1.737425
 9   0.596737
 Name: speed, dtype: float64, 0   -1.000000
 1    1.160000
 2    0.520000
 3    1.200000
 4    1.110000
 5    0.520000
 6    1.290000
 7    1.000000
 8    1.560000
 9    0.320000
 Name: sensed_speed, dtype: float64)

In [58]:
with_speeds_df_nona = with_speeds_df.dropna()

In [59]:
with_speeds_df.speed

0   0.000000
1   5.915331
2   0.790988
3   1.584267
4   1.547820
5   1.094819
6   0.657234
7   1.145274
8   1.737425
9   0.596737
Name: speed, dtype: float64

In [60]:
with_speeds_df.speed.median()

1.1200463971400958

In [61]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, hbtrip.start_place)

In [62]:
[loc_entry_list[0].data.loc.coordinates], start_place.data.location.coordinates

([[-122.0826288376279, 37.39067786774222]],
 [-122.0861964953338, 37.39118427507336])

In [63]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

320.177270563


speed ($s$) is in m/s. dist ($d$) is in meters.
so what we really need is

| Distance (m) | Time (s) | 
| --- | ---- |
| $s$ m | 1 sec |
| $d$ m | ? secs = $\frac{d}{s}$ |

In [64]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

285.86072093135624

In [65]:
(7 * 60  + add_time) / 60

11.764345348855937

In [66]:
356 + add_dist

676.1772705630359

### Closer to the real values (737m/13 mins), unsure if close enough

## Hooking up the results to the standard data structure

In [67]:
ts_delta = add_dist * with_speeds_df.speed.median()

In [68]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456198902.085297,2016-02-22T19:41:42.085297-08:00,37.390678,-122.082629
1,1456198905.000030,2016-02-22T19:41:45.000030-08:00,37.390633,-122.082816
2,1456198969.000034,2016-02-22T19:42:49.000034-08:00,37.390403,-122.082321
3,1456199001.000033,2016-02-22T19:43:21.000033-08:00,37.390838,-122.082146
4,1456199034.000033,2016-02-22T19:43:54.000033-08:00,37.391257,-122.081912
5,1456199080.000033,2016-02-22T19:44:40.000033-08:00,37.391596,-122.081534
6,1456199158.000033,2016-02-22T19:45:58.000033-08:00,37.391534,-122.080959
7,1456199202.000034,2016-02-22T19:46:42.000034-08:00,37.391635,-122.080403
8,1456199231.000033,2016-02-22T19:47:11.000033-08:00,37.392064,-122.080217
9,1456199315.000033,2016-02-22T19:48:35.000033-08:00,37.392359,-122.079788


In [69]:
loc_entry_df.columns

Index([              u'_id',          u'accuracy',          u'altitude',
                  u'filter',             u'floor',          u'fmt_time',
                 u'heading',          u'latitude',               u'loc',
            u'local_dt_day',     u'local_dt_hour',   u'local_dt_minute',
          u'local_dt_month',   u'local_dt_second', u'local_dt_timezone',
        u'local_dt_weekday',     u'local_dt_year',         u'longitude',
       u'metadata_write_ts',      u'sensed_speed',                u'ts',
                 u'user_id',         u'vaccuracy'],
      dtype='object')

In [70]:
start_place_enter_loc_entry = ecwe.Entry(ts.get_entry_at_ts("background/filtered_location", "data.ts", start_place.data.enter_ts))

In [71]:
start_place_enter_loc_entry

Entry({u'user_id': UUID('079e0f1a-c440-3d7c-b0e7-de160f748e35'), u'_id': ObjectId('56cbcb4ceaedff78c763669e'), u'data': {u'loc': {u'type': u'Point', u'coordinates': [-122.0861964953338, 37.39118427507336]}, u'fmt_time': u'2016-02-22T16:56:58.000047-08:00', u'altitude': 19.34386975527782, u'floor': 0, u'vaccuracy': 3, u'ts': 1456189018.000047, u'longitude': -122.0861964953338, u'filter': u'distance', u'local_dt': {u'hour': 16, u'month': 2, u'second': 58, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 56}, u'latitude': 37.39118427507336, u'heading': 206.015625, u'sensed_speed': 9.09000015258789, u'accuracy': 5}, u'metadata': {u'write_fmt_time': u'2016-02-22T16:56:58.174159-08:00', u'plugin': u'none', u'write_ts': 1456189018.174159, u'time_zone': u'America/Los_Angeles', u'platform': u'ios', u'write_local_dt': {u'hour': 16, u'month': 2, u'second': 58, u'weekday': 0, u'year': 2016, u'timezone': u'America/Los_Angeles', u'day': 22, u'minute': 56}, u'

In [72]:
loc_entry_list.insert(0, start_place_enter_loc_entry)

In [73]:
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456189018.000047,2016-02-22T16:56:58.000047-08:00,37.391184,-122.086196
1,1456198902.085297,2016-02-22T19:41:42.085297-08:00,37.390678,-122.082629
2,1456198905.000030,2016-02-22T19:41:45.000030-08:00,37.390633,-122.082816
3,1456198969.000034,2016-02-22T19:42:49.000034-08:00,37.390403,-122.082321
4,1456199001.000033,2016-02-22T19:43:21.000033-08:00,37.390838,-122.082146
5,1456199034.000033,2016-02-22T19:43:54.000033-08:00,37.391257,-122.081912
6,1456199080.000033,2016-02-22T19:44:40.000033-08:00,37.391596,-122.081534
7,1456199158.000033,2016-02-22T19:45:58.000033-08:00,37.391534,-122.080959
8,1456199202.000034,2016-02-22T19:46:42.000034-08:00,37.391635,-122.080403
9,1456199231.000033,2016-02-22T19:47:11.000033-08:00,37.392064,-122.080217


In [74]:
new_start_ts = loc_entry_df.loc[1].ts - ts_delta
new_start_ts, arrow.get(new_start_ts).to("America/Los_Angeles")

(1456198543.4718988, <Arrow [2016-02-22T19:35:43.471899-08:00]>)

In [75]:
loc_entry_df.ts.iloc[0] = new_start_ts
loc_entry_df.fmt_time.iloc[0] = arrow.get(new_start_ts).to(start_place_enter_loc_entry.data.local_dt.timezone)

/Users/shankari/OSS/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [76]:
loc_entry_df

,_id,accuracy,altitude,filter,floor,fmt_time,heading,latitude,loc,local_dt_day,...,local_dt_second,local_dt_timezone,local_dt_weekday,local_dt_year,longitude,metadata_write_ts,sensed_speed,ts,user_id,vaccuracy
0,56cbcb4ceaedff78c763669e,5,19.343870,distance,0,2016-02-22T19:35:43.471899-08:00,206.015625,37.391184,"{u'type': u'Point', u'coordinates': [-122.0861...",22,...,58,America/Los_Angeles,0,2016,-122.086196,1456189018.174159,9.090000,1456198543.471899,079e0f1a-c440-3d7c-b0e7-de160f748e35,3.000000
1,56cbf570eaedff78c7636a48,65,29.636635,distance,0,2016-02-22T19:41:42.085297-08:00,-1.000000,37.390678,"{u'type': u'Point', u'coordinates': [-122.0826...",22,...,42,America/Los_Angeles,0,2016,-122.082629,1456198903.092138,-1.000000,1456198902.085297,079e0f1a-c440-3d7c-b0e7-de160f748e35,19.940405
2,56cbf570eaedff78c7636a4c,10,29.574341,distance,0,2016-02-22T19:41:45.000030-08:00,273.515625,37.390633,"{u'type': u'Point', u'coordinates': [-122.0828...",22,...,45,America/Los_Angeles,0,2016,-122.082816,1456198905.121114,1.160000,1456198905.000030,079e0f1a-c440-3d7c-b0e7-de160f748e35,4.000000
3,56cbf570eaedff78c7636a52,10,27.620148,distance,0,2016-02-22T19:42:49.000034-08:00,128.320312,37.390403,"{u'type': u'Point', u'coordinates': [-122.0823...",22,...,49,America/Los_Angeles,0,2016,-122.082321,1456198969.145697,0.520000,1456198969.000034,079e0f1a-c440-3d7c-b0e7-de160f748e35,4.000000
4,56cbf570eaedff78c7636a55,10,26.250458,distance,0,2016-02-22T19:43:21.000033-08:00,67.500000,37.390838,"{u'type': u'Point', u'coordinates': [-122.0821...",22,...,21,America/Los_Angeles,0,2016,-122.082146,1456199001.161527,1.200000,1456199001.000033,079e0f1a-c440-3d7c-b0e7-de160f748e35,4.000000
5,56cbf571eaedff78c7636a5a,5,27.534332,distance,0,2016-02-22T19:43:54.000033-08:00,46.757812,37.391257,"{u'type': u'Point', u'coordinates': [-122.0819...",22,...,54,America/Los_Angeles,0,2016,-122.081912,1456199034.163653,1.110000,1456199034.000033,079e0f1a-c440-3d7c-b0e7-de160f748e35,4.000000
6,56cbf571eaedff78c7636a60,10,24.737732,distance,0,2016-02-22T19:44:40.000033-08:00,39.726562,37.391596,"{u'type': u'Point', u'coordinates': [-122.0815...",22,...,40,America/Los_Angeles,0,2016,-122.081534,1456199080.137458,0.520000,1456199080.000033,079e0f1a-c440-3d7c-b0e7-de160f748e35,4.000000
7,56cbf571eaedff78c7636a64,10,26.791779,distance,0,2016-02-22T19:45:58.000033-08:00,94.921875,37.391534,"{u'type': u'Point', u'coordinates': [-122.0809...",22,...,58,America/Los_Angeles,0,2016,-122.080959,1456199158.158537,1.290000,1456199158.000033,079e0f1a-c440-3d7c-b0e7-de160f748e35,4.000000
8,56cbf571eaedff78c7636a67,5,24.615936,distance,0,2016-02-22T19:46:42.000034-08:00,49.218750,37.391635,"{u'type': u'Point', u'coordinates': [-122.0804...",22,...,42,America/Los_Angeles,0,2016,-122.080403,1456199202.161730,1.000000,1456199202.000034,079e0f1a-c440-3d7c-b0e7-de160f748e35,3.000000
9,56cbf571eaedff78c7636a6b,5,23.957642,distance,0,2016-02-22T19:47:11.000033-08:00,20.390625,37.392064,"{u'type': u'Point', u'coordinates': [-122.0802...",22,...,11,America/Los_Angeles,0,2016,-122.080217,1456199231.185850,1.560000,1456199231.000033,079e0f1a-c440-3d7c-b0e7-de160f748e35,3.000000


## Sanity testing some longer trips (to and from Stevenson in the afternoon)

### To Stevenson (around 1pm)

In [77]:
trip = trips_df.iloc[3] 

In [78]:
trip_id = trip._id

In [79]:
len(esdt.get_raw_sections_for_trip(curr_id, trip_id))

1

In [80]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, trip_id)[0]

In [81]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.1075074003284, 37.41650822575895], "type": "Point"},
 u'2016-02-22T16:49:30.999990-08:00')

In [82]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [83]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456188570.999990,2016-02-22T16:49:30.999990-08:00,37.416508,-122.107507
1,1456188584.999991,2016-02-22T16:49:44.999991-08:00,37.416215,-122.107048
2,1456188599.999991,2016-02-22T16:49:59.999991-08:00,37.416502,-122.106603
3,1456188608.999991,2016-02-22T16:50:08.999991-08:00,37.416960,-122.106372
4,1456188617.999991,2016-02-22T16:50:17.999991-08:00,37.417360,-122.106064
5,1456188634.999991,2016-02-22T16:50:34.999991-08:00,37.417269,-122.105452
6,1456188639.999991,2016-02-22T16:50:39.999991-08:00,37.417048,-122.104952
7,1456188646.999991,2016-02-22T16:50:46.999991-08:00,37.416786,-122.104473
8,1456188682.999990,2016-02-22T16:51:22.999990-08:00,37.416511,-122.103954
9,1456188687.999991,2016-02-22T16:51:27.999991-08:00,37.416253,-122.103464


In [84]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(loc_entry_df)])])

Found 87 features from 87 points


In [85]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [86]:
with_speeds_df.speed, with_speeds_df.sensed_speed

(0     0.000000
 1     3.716771
 2     3.376557
 3     6.099175
 4     5.789376
 5     3.234787
 6    10.090307
 7     7.340571
 8     1.530999
 9    10.377795
 10   14.635526
 11   12.252217
 12   10.786900
 13   11.529778
 14   12.092465
 15   13.220330
 16   15.035835
 17   15.686972
 18   16.038245
 19   16.063559
 20   15.541513
 21   15.369687
 22   15.256784
 23   15.467901
 24   15.264925
 25   14.978938
 26   15.439729
 27   15.868718
 28   15.804112
 29   16.129860
         ...   
 57   13.672193
 58   13.280280
 59   10.168399
 60    5.275285
 61   10.324750
 62   11.749164
 63   11.813405
 64   11.958951
 65   12.784115
 66   11.468115
 67   11.776243
 68   12.651063
 69   12.659975
 70   12.184777
 71   12.191412
 72   12.338385
 73   12.977288
 74   14.158596
 75   15.015400
 76   14.692795
 77   14.352996
 78   13.410037
 79   13.131630
 80   12.959032
 81   13.391600
 82   14.838791
 83   15.118285
 84   14.109099
 85   13.623716
 86   11.366803
 Name: speed, dtype: flo

In [87]:
with_speeds_df_nona = with_speeds_df.dropna()

In [88]:
with_speeds_df.speed

0     0.000000
1     3.716771
2     3.376557
3     6.099175
4     5.789376
5     3.234787
6    10.090307
7     7.340571
8     1.530999
9    10.377795
10   14.635526
11   12.252217
12   10.786900
13   11.529778
14   12.092465
15   13.220330
16   15.035835
17   15.686972
18   16.038245
19   16.063559
20   15.541513
21   15.369687
22   15.256784
23   15.467901
24   15.264925
25   14.978938
26   15.439729
27   15.868718
28   15.804112
29   16.129860
        ...   
57   13.672193
58   13.280280
59   10.168399
60    5.275285
61   10.324750
62   11.749164
63   11.813405
64   11.958951
65   12.784115
66   11.468115
67   11.776243
68   12.651063
69   12.659975
70   12.184777
71   12.191412
72   12.338385
73   12.977288
74   14.158596
75   15.015400
76   14.692795
77   14.352996
78   13.410037
79   13.131630
80   12.959032
81   13.391600
82   14.838791
83   15.118285
84   14.109099
85   13.623716
86   11.366803
Name: speed, dtype: float64

In [89]:
with_speeds_df.speed.median()

13.410037358528445

In [90]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, trip.start_place)

In [91]:
[loc_entry_list[0].data.loc.coordinates], start_place.data.location.coordinates

([[-122.1075074003284, 37.41650822575895]],
 [-122.1080831532572, 37.41654267538098])

In [92]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

50.9919317646


speed ($s$) is in m/s. dist ($d$) is in meters.
so what we really need is

| Distance (m) | Time (s) | 
| --- | ---- |
| $s$ m | 1 sec |
| $d$ m | ? secs = $\frac{d}{s}$ |

In [93]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

3.8025197396038335

In [94]:
section_entry.data.duration/60, (section_entry.data.duration  + add_time) / 60

(7.450000949700674, 7.5133762786940705)

In [95]:
with_speeds_df_nona.distance.sum(), with_speeds_df_nona.distance.sum() + add_dist

(4903.3230143310193, 4954.3149460956483)

In [96]:
arrow.get(start_place.data.exit_ts - add_time).to("America/Los_Angeles")

<Arrow [2016-02-22T16:49:27.197470-08:00]>

### From Stevenson (around 3:30pm)

In [97]:
trip = trips_df.iloc[4] 

In [98]:
trip_id = trip._id

In [99]:
len(esdt.get_raw_sections_for_trip(curr_id, trip_id))

1

In [100]:
section_entry = esdt.get_raw_sections_for_trip(curr_id, trip_id)[0]

In [101]:
section_entry.data.start_loc, section_entry.data.start_fmt_time

({"coordinates": [-122.0826288376279, 37.39067786774222], "type": "Point"},
 u'2016-02-22T19:41:42.085297-08:00')

In [102]:
ts = esta.TimeSeries.get_time_series(curr_id)

In [103]:
loc_entry_it = ts.find_entries(["background/filtered_location"],
                                   esda.get_time_query_for_trip_like(
                                       esda.RAW_SECTION_KEY, section_entry.get_id()))
loc_entry_list = [ecwe.Entry(e) for e in loc_entry_it]
loc_entry_df = estb.BuiltinTimeSeries.to_data_df("background/filtered_location", loc_entry_list)
loc_entry_df[["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
0,1456198902.085297,2016-02-22T19:41:42.085297-08:00,37.390678,-122.082629
1,1456198905.000030,2016-02-22T19:41:45.000030-08:00,37.390633,-122.082816
2,1456198969.000034,2016-02-22T19:42:49.000034-08:00,37.390403,-122.082321
3,1456199001.000033,2016-02-22T19:43:21.000033-08:00,37.390838,-122.082146
4,1456199034.000033,2016-02-22T19:43:54.000033-08:00,37.391257,-122.081912
5,1456199080.000033,2016-02-22T19:44:40.000033-08:00,37.391596,-122.081534
6,1456199158.000033,2016-02-22T19:45:58.000033-08:00,37.391534,-122.080959
7,1456199202.000034,2016-02-22T19:46:42.000034-08:00,37.391635,-122.080403
8,1456199231.000033,2016-02-22T19:47:11.000033-08:00,37.392064,-122.080217
9,1456199315.000033,2016-02-22T19:48:35.000033-08:00,37.392359,-122.079788


In [104]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(loc_entry_df)])])

Found 10 features from 10 points


In [105]:
with_speeds_df = eaicl.add_dist_heading_speed(loc_entry_df)

In [106]:
with_speeds_df.speed, with_speeds_df.sensed_speed

(0   0.000000
 1   5.915331
 2   0.790988
 3   1.584267
 4   1.547820
 5   1.094819
 6   0.657234
 7   1.145274
 8   1.737425
 9   0.596737
 Name: speed, dtype: float64, 0   -1.000000
 1    1.160000
 2    0.520000
 3    1.200000
 4    1.110000
 5    0.520000
 6    1.290000
 7    1.000000
 8    1.560000
 9    0.320000
 Name: sensed_speed, dtype: float64)

In [107]:
with_speeds_df_nona = with_speeds_df.dropna()

In [108]:
with_speeds_df.speed

0   0.000000
1   5.915331
2   0.790988
3   1.584267
4   1.547820
5   1.094819
6   0.657234
7   1.145274
8   1.737425
9   0.596737
Name: speed, dtype: float64

In [109]:
with_speeds_df.speed.median()

1.1200463971400958

In [110]:
start_place = ts.get_entry_from_id(esda.RAW_PLACE_KEY, trip.start_place)

In [111]:
[loc_entry_list[0].data.loc.coordinates], start_place.data.location.coordinates

([[-122.0826288376279, 37.39067786774222]],
 [-122.0861964953338, 37.39118427507336])

In [112]:
add_dist = ecc.calDistance(loc_entry_list[0].data.loc.coordinates, start_place.data.location.coordinates)
print add_dist

320.177270563


speed ($s$) is in m/s. dist ($d$) is in meters.
so what we really need is

| Distance (m) | Time (s) | 
| --- | ---- |
| $d$ = 11.24 m | 1 sec |
| $d$ = 875.28 m | ? secs = $\frac{d}{s}$ $\approx$ 77|

In [113]:
add_time = add_dist / with_speeds_df.speed.median()
add_time

285.86072093135624

In [114]:
section_entry.data.duration/60, (section_entry.data.duration  + add_time) / 60

(6.881912263234456, 11.646257612090393)

In [115]:
with_speeds_df_nona.distance.sum(), with_speeds_df_nona.distance.sum() + add_dist

(422.16858730148215, 742.34585786451805)

In [116]:
arrow.get(start_place.data.exit_ts - add_time).to("America/Los_Angeles")

<Arrow [2016-02-22T19:36:56.224576-08:00]>

### Hm, the new duration after everything is just 2.9 $\approx$ 3 mins! Seems way too short compared to the trip to Stevenson

It is because in the trip to Stevenson, although we arrive at the campus by 13:08, we continue to get points until 13:13 as we walk to the classroom. But here, points end at 15:37 and then there's a large gap until we leave for karate at 15:49-ish. If we had got a couple of more points, we would be up there on this too. The bigger issue is that we don't have points for half the trip on each side, and while going to school, the missed points are mostly on a straight line down Shoreline while coming back from school, the missed points are on the Wright curve.

In [117]:
loc_entry_df = ts.get_data_df("background/filtered_location", time_query=None)
loc_entry_df[(loc_entry_df.local_dt_hour==15) & (loc_entry_df.local_dt_minute < 55)][["ts", "fmt_time", "latitude", "longitude"]]

,ts,fmt_time,latitude,longitude
27,1456183998.550963,2016-02-22T15:33:18.550963-08:00,37.403128,-122.084823
28,1456184136.000126,2016-02-22T15:35:36.000126-08:00,37.396229,-122.083001
29,1456184145.000135,2016-02-22T15:35:45.000135-08:00,37.395792,-122.083205
30,1456184185.000135,2016-02-22T15:36:25.000135-08:00,37.395452,-122.083622
31,1456184191.000135,2016-02-22T15:36:31.000135-08:00,37.394931,-122.083867
32,1456184196.000135,2016-02-22T15:36:36.000135-08:00,37.394518,-122.084095
33,1456184201.000136,2016-02-22T15:36:41.000136-08:00,37.394087,-122.084345
34,1456184205.000135,2016-02-22T15:36:45.000135-08:00,37.393645,-122.084594
35,1456184209.000136,2016-02-22T15:36:49.000136-08:00,37.393208,-122.084878
36,1456184213.000134,2016-02-22T15:36:53.000134-08:00,37.392771,-122.085274


### Zero speed investigation

Load data for:
- iphone_2016-02-22
- shankari_2016-06-20

In [118]:
from uuid import UUID

In [119]:
query = {'$or': [{'metadata.key': 'background/filtered_location'}], 'user_id': UUID('079e0f1a-c440-3d7c-b0e7-de160f748e35'), 'data.ts': {'$lte': 1456199315.000033, '$gte': 1456198902.085297}}

In [120]:
entries = [ecwe.Entry(e) for e in list(edb.get_timeseries_db().find(query).sort("data.ts"))]
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in entries])
loc_entry_df[["ts", "latitude", "longitude", "metadata_write_ts", "_id"]]

,ts,latitude,longitude,metadata_write_ts,_id
0,1456198902.085297,37.390678,-122.082629,1456198903.092138,56cbf570eaedff78c7636a48
1,1456198902.085297,37.390678,-122.082629,1456198903.092138,56cbd94beaedff78c7636755
2,1456198905.000030,37.390633,-122.082816,1456198905.121114,56cbf570eaedff78c7636a4c
3,1456198905.000030,37.390633,-122.082816,1456198905.121114,56cbd94beaedff78c7636757
4,1456198969.000034,37.390403,-122.082321,1456198969.145697,56cbf570eaedff78c7636a52
5,1456198969.000034,37.390403,-122.082321,1456198969.145697,56cbd94beaedff78c7636759
6,1456199001.000033,37.390838,-122.082146,1456199001.161527,56cbf570eaedff78c7636a55
7,1456199001.000033,37.390838,-122.082146,1456199001.161527,56cbd94beaedff78c763675b
8,1456199034.000033,37.391257,-122.081912,1456199034.163653,56cbf571eaedff78c7636a5a
9,1456199034.000033,37.391257,-122.081912,1456199034.163653,56cbd94beaedff78c763675d


In [121]:
deduped_loc_entry_df = loc_entry_df.drop_duplicates(subset=["ts", "metadata_write_ts", "latitude", "longitude", "altitude", "accuracy"])
deduped_loc_entry_df[["ts", "latitude", "longitude", "metadata_write_ts", "_id"]]

,ts,latitude,longitude,metadata_write_ts,_id
0,1456198902.085297,37.390678,-122.082629,1456198903.092138,56cbf570eaedff78c7636a48
2,1456198905.000030,37.390633,-122.082816,1456198905.121114,56cbf570eaedff78c7636a4c
4,1456198969.000034,37.390403,-122.082321,1456198969.145697,56cbf570eaedff78c7636a52
6,1456199001.000033,37.390838,-122.082146,1456199001.161527,56cbf570eaedff78c7636a55
8,1456199034.000033,37.391257,-122.081912,1456199034.163653,56cbf571eaedff78c7636a5a
10,1456199080.000033,37.391596,-122.081534,1456199080.137458,56cbf571eaedff78c7636a60
12,1456199158.000033,37.391534,-122.080959,1456199158.158537,56cbf571eaedff78c7636a64
14,1456199202.000034,37.391635,-122.080403,1456199202.161730,56cbf571eaedff78c7636a67
16,1456199231.000033,37.392064,-122.080217,1456199231.185850,56cbf571eaedff78c7636a6b
18,1456199315.000033,37.392359,-122.079788,1456199315.157882,56cbf571eaedff78c7636a70


In [122]:
with_speeds_df = eaicl.add_dist_heading_speed(deduped_loc_entry_df)
with_speeds_df.speed.median()

1.1200463971400958

In [123]:
from uuid import UUID

In [124]:
query = {'$or': [{'metadata.key': 'background/filtered_location'}], 'user_id': UUID('0763de67f61e3f5d90e7518e69793954'), 'metadata.write_ts': {'$lte': 1472185672.217289}}

In [125]:
entries = [ecwe.Entry(e) for e in list(edb.get_timeseries_db().find(query).sort("data.ts"))]
loc_entry_df = pd.DataFrame([ts._to_df_entry(e) for e in entries])
loc_entry_df[["ts", "latitude", "longitude", "metadata_write_ts", "_id"]].head()

,ts,latitude,longitude,metadata_write_ts,_id
0,1466436483.395000,37.389805,-122.085786,1466436497.047000,576810cb4a9bba50e3fa1d56
1,1466436483.395000,37.389805,-122.085786,1466436497.047000,57681ee64a9bba50e3fa1fe0
2,1466436506.530000,37.389532,-122.085291,1466436508.614000,576810cb4a9bba50e3fa1d59
3,1466436506.530000,37.389532,-122.085291,1466436508.614000,57681ee64a9bba50e3fa1fe3
4,1466436515.231000,37.389701,-122.085216,1466436530.006000,576810cb4a9bba50e3fa1d5c


In [126]:
deduped_loc_entry_df = loc_entry_df.drop_duplicates(subset=["ts", "metadata_write_ts", "latitude", "longitude", "altitude", "accuracy"])
deduped_loc_entry_df[["ts", "latitude", "longitude", "metadata_write_ts", "_id"]].head()

,ts,latitude,longitude,metadata_write_ts,_id
0,1466436483.395000,37.389805,-122.085786,1466436497.047000,576810cb4a9bba50e3fa1d56
2,1466436506.530000,37.389532,-122.085291,1466436508.614000,576810cb4a9bba50e3fa1d59
4,1466436515.231000,37.389701,-122.085216,1466436530.006000,576810cb4a9bba50e3fa1d5c
6,1466436546.031000,37.389305,-122.084656,1466436546.833000,576810cb4a9bba50e3fa1d5f
8,1466436553.963000,37.389305,-122.084656,1466436561.020000,576810cb4a9bba50e3fa1d62


## Dataframe testing

In [127]:
test_df = pd.DataFrame({"A": [1,2,3], "B": [11,12,13]})
test_df

,A,B
0,1,11
1,2,12
2,3,13


In [128]:
test_df.loc[-1] = {'B': 10, 'A': 0}
test_df

,A,B
0,1,11
1,2,12
2,3,13
-1,0,10


In [129]:
test_df.sort_index().reset_index(drop=True)

,A,B
0,0,10
1,1,11
2,2,12
3,3,13


In [130]:
test_df.drop(["B"], axis=1)

,A
0,1
1,2
2,3
-1,0


In [131]:
valid = [1,2]
test_df[np.logical_not(test_df.A.isin(valid))]

,A,B
2,3,13
-1,0,10
